In [31]:
#load environment variables
import os
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
from langchain_core.tools import tool
from langchain_core.runnables import RunnableParallel

load_dotenv()

llm = OllamaLLM(model="mistral:latest", temperature=0)
llm.invoke("Me puedes decir cuanto es 4*56")

'4 * 56 = 224\n\nEn español: 4 veces 56 es 224.'

In [6]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dos números enteros."""
    return a * b

llm_with_tools = RunnableParallel(
    {"llm": llm, "multiply": multiply}  # Usar directamente la función decorada
)

response = llm_with_tools.invoke("Me puedes decir cuánto es 4*56")
print(response)

ValidationError: 2 validation errors for multiply
a
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Me puedes decir cuánto es 4*56', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing
b
  Field required [type=missing, input_value={'a': 'Me puedes decir cuánto es 4*56'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [12]:
# Cargar variables de entorno
import os
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
from langchain_core.tools import tool
from langchain_core.runnables import RunnableParallel

load_dotenv()

# Definir la herramienta correctamente con el decorador @tool
@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dos números enteros."""
    return a * b

# Configurar el modelo de lenguaje
llm = OllamaLLM(
    model="deepseek-r1:latest",
    temperature=0,
    system="Si necesitas hacer cálculos matemáticos, usa la herramienta de multiplicación."
)


# Configurar la ejecución con herramientas
llm_with_tools = RunnableParallel(
    {"llm": llm, "multiply": multiply}  # Correcto: pasa la función decorada como herramienta
)

print(llm_with_tools)

response = llm_with_tools.invoke("Calcula 4 por 56 usando la herramienta de multiplicación.")

# Invocar el modelo con herramientas activadas
#response = llm_with_tools.invoke("Me puedes decir cuánto es 4*56")
#print(response)


steps__={'llm': OllamaLLM(model='deepseek-r1:latest', temperature=0.0), 'multiply': StructuredTool(name='multiply', description='Multiplica dos números enteros.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x77f3792163e0>)}


ValidationError: 2 validation errors for multiply
a
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Calcula 4 por 56 usando ...nta de multiplicación.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing
b
  Field required [type=missing, input_value={'a': 'Calcula 4 por 56 u...ta de multiplicación.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [24]:
# Cargar variables de entorno
import os
import re
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
from langchain_core.tools import tool
from langchain_core.runnables import RunnableParallel

load_dotenv()

# Definir la herramienta correctamente con el decorador @tool
@tool
def multiply(query: str) -> int:
    """Extrae dos números de una consulta en texto y los multiplica."""
    numbers = [int(num) for num in re.findall(r'\d+', query)]
    if len(numbers) == 2:
        return numbers[0] * numbers[1]
    return "Error: No se encontraron dos números en la consulta."

# Configurar el modelo de lenguaje con herramientas
llm = OllamaLLM(
    model="mistral:latest",  # Puedes probar con otros modelos como gemma:latest o llama3:latest
    temperature=0,
    system="Si necesitas hacer cálculos matemáticos, usa la herramienta de multiplicación."
)

# Vincular herramientas usando RunnableParallel
llm_with_tools = RunnableParallel(
    {"llm": llm, "multiply": multiply}
)

# Verificar que la herramienta está registrada correctamente
print(llm_with_tools)

# Probar invocación
response = llm_with_tools.invoke("Calcula 4 por 56 usando la herramienta de multiplicación.")

# Imprimir la respuesta en formato JSON para ver si la herramienta se usa correctamente
import json
print(json.dumps(response, indent=4, ensure_ascii=False))


steps__={'llm': OllamaLLM(model='mistral:latest', temperature=0.0), 'multiply': StructuredTool(name='multiply', description='Extrae dos números de una consulta en texto y los multiplica.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x77f36ec28540>)}
{
    "llm": "4 x 56 = 224\n\nEn este caso, puedes multiplicar los números uno a uno:\n\n- Primero, multiplica 4 por 6 (4 * 6 = 24)\n- Después, multiplica el resultado por 5 (24 * 5 = 120)\n- Finalmente, multiplica el resultado por 1 (120 * 1 = 120)\n\nPero es más fácil usar la herramienta de multiplicación:\n\n- Empieza en la columna de los números mayores y sigue hacia la izquierda hasta llegar a la columna de los números menores.\n- En cada intersección, si hay un número en ambas columnas, entonces multiplica ese número por el número que está debajo de él en la columna de los números mayores y suma el resultado a la fila correspondiente en la tabla de multiplicar.\n- En este caso, 4 está en l

In [25]:
# Cargar variables de entorno
import os
import re
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_openai_functions_agent

load_dotenv()

# Definir la herramienta correctamente con el decorador @tool
@tool
def multiply(query: str) -> str:
    """Extrae dos números de una consulta en texto y los multiplica."""
    numbers = [int(num) for num in re.findall(r'\d+', query)]
    if len(numbers) == 2:
        return f"{numbers[0] * numbers[1]}"
    return "Error: No se encontraron dos números en la consulta."

# Configurar el modelo de lenguaje con instrucciones claras
llm = OllamaLLM(
    model="mistral:latest",  # Puedes probar con otros modelos como gemma:latest o llama3:latest
    temperature=0,
    system=(
        "Si necesitas hacer cálculos matemáticos, usa exclusivamente la herramienta de multiplicación.\n"
        "No intentes resolver cálculos por tu cuenta.\n"
        "Siempre delega cualquier operación matemática a la herramienta disponible.\n"
    )
)

# Crear un agente que use el LLM y la herramienta
agent = create_openai_functions_agent(llm, [multiply])

# Crear un ejecutor de agente
agent_executor = AgentExecutor(agent=agent, tools=[multiply], verbose=True)

# Probar invocación
response = agent_executor.invoke({"input": "Calcula 4 por 56 usando la herramienta de multiplicación."})

# Imprimir la respuesta
import json
print(json.dumps(response, indent=4, ensure_ascii=False))


TypeError: create_openai_functions_agent() missing 1 required positional argument: 'prompt'

In [26]:
# Cargar variables de entorno
import os
import re
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

# Definir la herramienta correctamente con el decorador @tool
@tool
def multiply(query: str) -> str:
    """Extrae dos números de una consulta en texto y los multiplica."""
    numbers = [int(num) for num in re.findall(r'\d+', query)]
    if len(numbers) == 2:
        return f"{numbers[0] * numbers[1]}"
    return "Error: No se encontraron dos números en la consulta."

# Configurar el modelo de lenguaje con instrucciones claras
llm = OllamaLLM(
    model="mistral:latest",  # Puedes probar con otros modelos como gemma:latest o llama3:latest
    temperature=0,
    system=(
        "Si necesitas hacer cálculos matemáticos, usa exclusivamente la herramienta de multiplicación.\n"
        "No intentes resolver cálculos por tu cuenta.\n"
        "Siempre delega cualquier operación matemática a la herramienta disponible.\n"
    )
)

# Definir un prompt base para el agente
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente experto en matemáticas. Siempre usas la herramienta 'multiply' para multiplicar números."),
    ("human", "{input}")
])

# Crear un agente que use el LLM, el prompt y la herramienta
agent = create_openai_functions_agent(llm, prompt, [multiply])

# Crear un ejecutor de agente
agent_executor = AgentExecutor(agent=agent, tools=[multiply], verbose=True)

# Probar invocación
response = agent_executor.invoke({"input": "Calcula 4 por 56 usando la herramienta de multiplicación."})

# Imprimir la respuesta
import json
print(json.dumps(response, indent=4, ensure_ascii=False))


AttributeError: 'list' object has no attribute 'input_variables'

In [27]:
# Cargar variables de entorno
import os
import re
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, initialize_agent, AgentType
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

# Definir la herramienta correctamente con el decorador @tool
@tool
def multiply(query: str) -> str:
    """Extrae dos números de una consulta en texto y los multiplica."""
    numbers = [int(num) for num in re.findall(r'\d+', query)]
    if len(numbers) == 2:
        return f"{numbers[0] * numbers[1]}"
    return "Error: No se encontraron dos números en la consulta."

# Configurar el modelo de lenguaje con instrucciones claras
llm = OllamaLLM(
    model="mistral:latest",  # Puedes probar con otros modelos como gemma:latest o llama3:latest
    temperature=0,
    system=(
        "Si necesitas hacer cálculos matemáticos, usa exclusivamente la herramienta de multiplicación.\n"
        "No intentes resolver cálculos por tu cuenta.\n"
        "Siempre delega cualquier operación matemática a la herramienta disponible.\n"
    )
)

# Inicializar el agente con las herramientas
agent_executor = initialize_agent(
    tools=[multiply],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Modo basado en razonamiento
    verbose=True
)

# Probar invocación
response = agent_executor.invoke({"input": "Calcula 4 por 56 usando la herramienta de multiplicación."})

# Imprimir la respuesta
import json
print(json.dumps(response, indent=4, ensure_ascii=False))


/tmp/ipykernel_4340/2310706049.py:33: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(




> Entering new AgentExecutor chain...
 I need to use the multiply tool to calculate 4 times 56.
Action: multiply
Action Input: "4, 56"
Observation: 224
Thought: I now know the final answer.
Final Answer: The result of 4 multiplied by 56 is 224.

> Finished chain.
{
    "input": "Calcula 4 por 56 usando la herramienta de multiplicación.",
    "output": "The result of 4 multiplied by 56 is 224."
}


In [29]:
# Cargar variables de entorno
import os
import re
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
from langchain_core.tools import tool
from langgraph.graph import StateGraph
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from typing import TypedDict, List

load_dotenv()

# Definir la herramienta correctamente con el decorador @tool
@tool
def multiply(query: str) -> str:
    """Extrae dos números de una consulta en texto y los multiplica."""
    numbers = [int(num) for num in re.findall(r'\d+', query)]
    if len(numbers) == 2:
        return f"{numbers[0] * numbers[1]}"
    return "Error: No se encontraron dos números en la consulta."

# Definir el modelo de lenguaje con instrucciones claras
llm = OllamaLLM(
    model="mistral:latest",  # Puedes probar con otros modelos como gemma:latest o llama3:latest
    temperature=0,
    system=(
        "Eres un asistente experto en matemáticas.\n"
        "Si necesitas hacer cálculos matemáticos, usa exclusivamente la herramienta de multiplicación.\n"
        "No intentes resolver cálculos por tu cuenta.\n"
        "Siempre delega cualquier operación matemática a la herramienta disponible.\n"
    )
)

# Definir el estado del agente en LangGraph
class AgentState(TypedDict):
    messages: List

# Crear el agente ReAct en LangGraph (sin pasar `prompt`)
agent = create_react_agent(llm, [multiply])

# Construir el flujo de ejecución con LangGraph
workflow = StateGraph(AgentState)

# Agregar el nodo del agente al gráfico
workflow.add_node("agent", agent)

# Configurar la transición entre nodos (solo tenemos el agente por ahora)
workflow.set_entry_point("agent")

# Construir el ejecutor de LangGraph
app = workflow.compile()

# Probar invocación en LangGraph
response = app.invoke({"messages": [HumanMessage(content="Calcula 4 por 56 usando la herramienta de multiplicación.")]})

# Imprimir la respuesta
import json
print(json.dumps(response, indent=4, ensure_ascii=False))


AttributeError: 'OllamaLLM' object has no attribute 'bind_tools'